In [2]:
import numpy as np
from scipy.spatial.distance import pdist, squareform, cdist

In [24]:
def kernel_se(_X1, _X2, _hyp={'gain': 1, 'len': 1, 'noise': 1e-8}):
    hyp_gain = float(_hyp['gain']) ** 2
    hyp_len = 1 / float(_hyp['len'])

    pairwise_dists = cdist(_X2, _X2, 'euclidean') # (n,n) shape matrix
    K = hyp_gain * np.exp(-pairwise_dists ** 2 / (hyp_len**2)) # (n,n) shape matrix
    return K

def kdpp(_X, _k):
    # Select _n samples out of _X using K-DPP
    n, d = _X.shape[0], _X.shape[1]
    mid_dist = np.median(cdist(_X, _X, "euclidean")) # distance의 중앙값 : median((200,200))
    out, idx = np.zeros(shape=(_k,d)), [] 

    for i in range(_k):
        if i == 0: # init
            rand_idx = np.random.randint(n)
            idx.append(rand_idx) # start index
            out[i,:] = _X[rand_idx,:] # start vectors
        else:
            det_vals = np.zeros(n)
            for j in range(n):
                if j in idx:
                    det_vals[j] = -np.inf
                else:
                    idx_temp = idx.copy()
                    idx_temp.append(j)

                    X_curr = _X[idx_temp, :]
                    K = kernel_se(X_curr, X_curr, {'gain': 1, 'len': mid_dist, 'noise': 1e-4})
                    det_vals[j] = np.linalg.det(K) # 행렬식 Matrix Determinant
            max_idx = np.argmax(det_vals) # find argmax j
            idx.append(max_idx)
            out[i, :] = _X[max_idx, :] # append argmax det(L)
    return out, idx

In [25]:
# Data
n = 200
k = 10
x = np.random.rand(n, 2)

kdpp_out, _ = kdpp(_X=x, _k=k)

In [27]:
kdpp_out, _

(array([[0.23564706, 0.46917656],
        [0.99138086, 0.97803945],
        [0.98666893, 0.02303899],
        [0.02641741, 0.03177045],
        [0.12991481, 0.94974638],
        [0.5710212 , 0.0039967 ],
        [0.64201505, 0.97691464],
        [0.90308863, 0.57318393],
        [0.00304744, 0.59728216],
        [0.76916351, 0.24722354]]),
 [1, 30, 168, 56, 136, 6, 105, 117, 182, 51])